In [ ]:
!pip uninstall -y whisper
!pip install -U openai-whisper
!pip uninstall -y whisper openai-whisper
!pip install git+https://github.com/openai/whisper.git


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.5/800.5 kB 11.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 44.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 86.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 70.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 36.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.0 MB/s

In [ ]:
!pip install speechbrain
!pip install torchaudio speechbrain whisper openai
!pip install openai-whisper
!pip show openai-whisper
!pip install openai
!pip install --upgrade openai
!pip install -U openai-whisper
!pip install torch torchaudio speechbrain whisper openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 824.8/824.8 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.7/117.7 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 739.1/739.1 kB 38.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 1.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for whisper: filename=whisper-1.1.10-py3-none-any.whl size=41120 sha256=bc4e6319480966c60fe4c08272362ffef1a3e2434fb02c93fca6d818840de6f3
  Stored in directory: /root/.cache/pip/wheels/21/65/ee/4e6672aabfa486d3341a39a04f8f87c77e5156149299b5a7d0
Successfully built whisper
Name: openai-whisper
Version: 20240930
Summary: Robust Speech Recognition via Large-Scale Weak Supervision
Home-page: https://github.com/openai/whisper
Author: OpenAI
Author-email: 
License: MIT
Location: /usr/local/lib/python3.11/dist-packages
Requires: more-itertools, numba, numpy, tiktoken, torch, tqdm, triton
Required-by: 
   ━━━━━━━━━━━━━━━━━━

In [ ]:
import torch
import torchaudio
import torchaudio.transforms as T
import whisper
from openai import OpenAI
import numpy as np
from typing import Dict, Any
import json
import logging
from datetime import datetime

In [ ]:
class StressAnalyzer:
    def __init__(self, openai_api_key: str):
        """Initialize the Stress Analyzer with required models and configurations."""
        logging.basicConfig(level=logging.INFO)
        self.logger = logging.getLogger(__name__)

        # Initialize OpenAI client with new API
        self.client = OpenAI(api_key=openai_api_key)

        self.logger.info("Loading Whisper model...")
        self.whisper_model = whisper.load_model("base")

        # Audio processing parameters
        self.sample_rate = 16000
        self.n_mels = 128
        self.n_fft = 2048
        self.hop_length = 512

        # Initialize audio transforms
        self.mel_transform = T.MelSpectrogram(
            sample_rate=self.sample_rate,
            n_fft=self.n_fft,
            hop_length=self.hop_length,
            n_mels=self.n_mels,
            normalized=True
        )

        self.logger.info("Initialization complete!")

    def _extract_audio_features(self, waveform: torch.Tensor) -> Dict[str, float]:
        """Extract audio features for emotion analysis."""
        try:
            # Convert to mel spectrogram
            mel_spec = self.mel_transform(waveform)
            mel_spec = torch.log1p(mel_spec)

            # Calculate features
            temporal_features = {
                "energy": float(torch.mean(torch.sum(mel_spec, dim=1))),
                "energy_std": float(torch.std(torch.sum(mel_spec, dim=1))),
                "spectral_centroid": float(torch.mean(
                    torch.sum(mel_spec * torch.arange(self.n_mels, device=mel_spec.device).view(1, -1, 1), dim=1) /
                    (torch.sum(mel_spec, dim=1) + 1e-8)
                )),
                "zero_crossing_rate": float(torch.mean((waveform[..., :-1] * waveform[..., 1:] < 0).float()))
            }

            # Map features to emotions
            emotions = self._map_features_to_emotions(temporal_features)
            return emotions

        except Exception as e:
            self.logger.error(f"Error extracting features: {str(e)}")
            raise

    def _map_features_to_emotions(self, features: Dict[str, float]) -> Dict[str, float]:
        """Map audio features to emotion scores."""
        try:
            # Normalize features
            max_energy = 10.0
            max_spectral = float(self.n_mels)

            norm_features = {
                "energy": min(1.0, features["energy"] / max_energy),
                "energy_std": min(1.0, features["energy_std"] / max_energy),
                "spectral_centroid": min(1.0, features["spectral_centroid"] / max_spectral),
                "zero_crossing_rate": min(1.0, features["zero_crossing_rate"] * 2.0)
            }

            # Calculate emotion scores
            emotions = {
                "angry": 0.4 * norm_features["energy"] + 0.3 * norm_features["zero_crossing_rate"] +
                        0.3 * norm_features["energy_std"],
                "happy": 0.4 * norm_features["energy"] + 0.4 * norm_features["spectral_centroid"] +
                        0.2 * (1 - norm_features["energy_std"]),
                "sad": 0.4 * (1 - norm_features["energy"]) + 0.3 * (1 - norm_features["spectral_centroid"]) +
                       0.3 * norm_features["energy_std"],
                "neutral": 0.4 * (1 - norm_features["energy_std"]) + 0.3 * (1 - abs(0.5 - norm_features["energy"])) +
                          0.3 * (1 - abs(0.5 - norm_features["spectral_centroid"]))
            }

            # Normalize to sum to 1
            total = sum(emotions.values()) + 1e-8
            emotions = {k: round(v/total, 4) for k, v in emotions.items()}

            return emotions

        except Exception as e:
            self.logger.error(f"Error mapping emotions: {str(e)}")
            raise

    def _transcribe_audio(self, audio_path: str) -> Dict[str, Any]:
        """Transcribe audio using Whisper."""
        try:
            result = self.whisper_model.transcribe(audio_path)
            return {
                "text": result["text"],
                "segments": [
                    {
                        "text": segment["text"],
                        "start": round(segment["start"], 2),
                        "end": round(segment["end"], 2),
                    }
                    for segment in result["segments"]
                ]
            }
        except Exception as e:
            self.logger.error(f"Error in transcription: {str(e)}")
            raise

    def _analyze_stress(self, emotion_scores: Dict[str, float],
                       transcription: Dict[str, Any]) -> Dict[str, Any]:
        """Get stress analysis from GPT-4 using new OpenAI API."""
        try:
            prompt = {
                "emotion_scores": emotion_scores,
                "transcription": transcription["text"],
                "instructions": "Based on the emotional scores and transcribed speech, provide:"
                              "\n1. Stress level (1-7 scale)"
                              "\n2. Confidence score (0-1)"
                              "\n3. List of stress indicators"
                              "\n4. Recommendations for stress management"
                              "\nReturn the response in JSON format."
            }

            # Using new OpenAI API format
            response = self.client.chat.completions.create(
                model="gpt-4",
                messages=[
                    {"role": "system", "content": "You are an assistant designed to analyze the stress level in speech. Focus on identifying emotional intensity, such as regret, nostalgia, uneasiness, or conflict, and map it to a stress level from 1 to 7. Provide an explanation for the level Provide analysis in JSON format."},
                    {"role": "user", "content": json.dumps(prompt, indent=2)}
                ],
                temperature=0.7
            )

            # Extract content from new API response format
            result = response.choices[0].message.content
            return json.loads(result)

        except Exception as e:
            self.logger.error(f"Error in GPT analysis: {str(e)}")
            raise

    def analyze_stress(self, audio_path: str) -> Dict[str, Any]:
        """Perform complete stress analysis on audio file."""
        try:
            # Load and preprocess audio
            waveform, sr = torchaudio.load(audio_path)
            if sr != self.sample_rate:
                resampler = T.Resample(sr, self.sample_rate)
                waveform = resampler(waveform)
            if waveform.shape[0] > 1:
                waveform = torch.mean(waveform, dim=0, keepdim=True)

            # Extract features
            emotion_scores = self._extract_audio_features(waveform)
            self.logger.info("Audio features extracted successfully")

            # Get transcription
            transcription = self._transcribe_audio(audio_path)
            self.logger.info("Transcription complete")

            # Get stress analysis
            stress_analysis = self._analyze_stress(emotion_scores, transcription)
            self.logger.info("Stress analysis complete")

            output = {
                "metadata": {
                    "timestamp": datetime.now().isoformat(),
                    "audio_file": audio_path,
                    "status": "success"
                },
                "analysis": {
                    "emotion_scores": emotion_scores,
                    "transcription": transcription,
                    "stress_assessment": stress_analysis
                }
            }

            self.logger.info("Analysis completed successfully")
            return output

        except Exception as e:
            self.logger.error(f"Error in stress analysis: {str(e)}")
            return {
                "metadata": {
                    "timestamp": datetime.now().isoformat(),
                    "audio_file": audio_path,
                    "status": "failed"
                },
                "error": str(e)
            }

def main():
    logging.basicConfig(
        level=logging.INFO,
        format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
    )
    logger = logging.getLogger(__name__)

    try:
        # Initialize analyzer with your OpenAI API key
        analyzer = StressAnalyzer(openai_api_key="hhhhhh")

        # Process audio file
        results = analyzer.analyze_stress("fire.mp3")
        results = analyzer.analyze_stress("sad.wav")
        results = analyzer.analyze_stress("New Recording 20.wav")
        results = analyzer.analyze_stress("New Recording 15.m4a")
        results = analyzer.analyze_stress("test1.mp3")
        results = analyzer.analyze_stress("New Recording 16.m4a")
        results = analyzer.analyze_stress("New Recording 13.m4a")
        results = analyzer.analyze_stress("New Recording 25.m4a")
        results = analyzer.analyze_stress("1008_IWW_ANG_XX.wav")
        #results = analyzer.analyze_stress("New Recording 26.m4a")
        results = analyzer.analyze_stress("test1.mp3")
        # Save results
        with open("stress_analysis_results.json", "w", encoding="utf-8") as f:
            json.dump(results, f, indent=2, ensure_ascii=False)

        print("Analysis complete. Results saved to stress_analysis_results.json")

        # Print key results
        if results["metadata"]["status"] == "success":
            analysis = results["analysis"]
            print("\nKey findings:")
            #print("Emotion Scores:", json.dumps(analysis["emotion_scores"], indent=2))
            print("Emotion Scores:", json.dumps(analysis["emotion_scores"], separators=(',', ':')))
            print("Transcription:", analysis["transcription"]["text"])
            print("Stress Assessment:", json.dumps(analysis["stress_assessment"], indent=2))
            #print("Stress Assessment:", json.dumps(analysis["stress_assessment"], separators=(',', ':')))
        else:
            print(f"Analysis failed: {results.get('error', 'Unknown error')}")

    except Exception as e:
        logger.error("Analysis failed", exc_info=True)
        print(f"Error during analysis: {str(e)}")

if __name__ == "__main__":
    main()

/usr/local/lib/python3.11/dist-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(fp, map_location=device)
ERROR:__main_

Analysis complete. Results saved to stress_analysis_results.json

Key findings:
Emotion Scores: {"angry":0.1278,"happy":0.1823,"sad":0.3041,"neutral":0.3858}
Transcription:  loans that we can offer with this or farm ownership loans, operating lines of credit, or equipment and capital improvement need loans. The benefit to the
Stress Assessment: {
  "stress_level": 2,
  "confidence_score": 0.85,
  "stress_indicators": [
    "Neutral tone",
    "Minor sadness"
  ],
  "recommendations": [
    "Maintain composure and calmness",
    "Ensure clear and positive communication"
  ],
  "explanation": "The speaker's speech is largely neutral with a minor indication of sadness. However, the overall emotional intensity is low, suggesting a low stress level. Therefore, the stress level is evaluated as 2 out of 7."
}
